In [1]:
from pprint import pprint
import xml.etree.ElementTree as ET
import re

#import requests as req 

In [2]:
# Reading file from request
HTML_f = open("new.xml.html", "r") 

# Reading the file 
html_t = HTML_f.read() 

# saving file as list

html_l = list(html_t.split('>'))

In [3]:
def flatten_str_l(input_l):
    res = []
    for i in input_l:
        if isinstance(i,list): res.extend(flatten(i))
        else: res.append(i)
    return res

In [4]:
def get_event_info(start_i,info):
    # filter out useful information for each event 
        e_id = info[start_i+15]
        e_title = info[start_i+17]
        e_type = info[start_i+22]
        e_group = info[start_i+24]
        e_time = info[start_i+27]
        e_type2 = info[start_i+30]
        #von-bis-days in [start_i+33::start_i+37]
        #room name in [start_i+40]
        #prof name in [start_i+61]
        event = [e_id, e_title, e_type, e_group, e_time, e_type2]
        return [x.replace('</span','').replace(', ','').strip()
                 for x in event if not 'undefined' in x]
    

In [21]:
# list of days (keywords), delete Sunday?
days = ["Montag","Dienstag", "Mittwoch", "Donnerstag", "Freitag", "Samstag", "Sonntag"]

res_dict = dict.fromkeys(days)
# get information for each day of the week 
for day in days:
    start = [i for i,x in enumerate(html_l) if day in x]
    if len(start) !=0:
        d = start[1]
        info = flatten_str_l(html_l[d::])
        
        #TODO: loop trough days
        # [96] second event for friday, 100 rest
        # Sa/So different [70]
        #print(day,info[70])
        events = []
        for i in range(0,len(info),81): #not working, find regex to match beginning of event
            if i+61 < len(info):
                new_ev = get_event_info(i, info)
                events.append(new_ev)
                res_dict[day]= events
            

# output-dictionary with objects, divided by day, ordered by time
pprint(res_dict['Freitag'])


[['EV-17-PRÜ-470',
  'Klausur Ecosystem Restoration',
  'interne Veranstaltung',
  '1. PG',
  '09:00 bis 11:00',
  'Einzeltermin'],
 ['EV-17-VOR-035',
  'Vorbereitungen zu LV-17-b01-026 (Bestimmungsübungen für Fortgeschrittene)',
  'interne Veranstaltung',
  '1. PG',
  '09:00 bis 17:00',
  'Einzeltermin'],
 ['<div class="scheduleTimes"',
  '09:00 bis 18:00',
  '<span '
  'id="plan:schedule:scheduleColumn:4:termin:2:scheduleItem:scheduleStartDate"',
  '<span '
  'id="plan:schedule:scheduleColumn:4:termin:2:scheduleItem:scheduleDateSeperator"',
  '30.08.2024',
  'BKR3 + BKR4'],
 ['<span '
  'id="plan:schedule:scheduleColumn:4:termin:3:scheduleItem:eventtypeShorttext"',
  '<span '
  'id="plan:schedule:scheduleColumn:4:termin:3:scheduleItem:parallelgroupshorttext"',
  '</div',
  'Blockveranstaltung',
  '16.09.2024',
  '<span '
  'id="plan:schedule:scheduleColumn:4:termin:3:scheduleItem:scheduleEndDate"'],
 ['Field Course of Mycology (Exkursion und Übung; 2 Wochen ganztags)',
  '<div class=